In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.append('/home/jakobs/Documents/IndEcol/OASES/ecospold2matrix/')
sys.path.append('/home/jakobs/Documents/IndEcol/OASES/pymrio/')
sys.path.append('/home/jakobs/Documents/IndEcol/OASES/pylcaio/src/')
import ecospold2matrix as e2m
import pymrio
import pylcaio

sys.path.append('/home/jakobs/Documents/IndEcol/OASES/Accuracy_vs_precission_HLCA/scripts')
from aggregate import build_MultiIndex_Aggregation_Matrix, Aggregation_dic_from_file


import os
import gzip
import pickle
import pandas as pd
import itertools

from importlib import reload
import pdb
import scipy.sparse

ModuleNotFoundError: No module named 'aggregate'

## Prepare Ecoinvent data with ecospold2matrix

In [ ]:
# %%time

# #run ecospold2matrix
# ecospold_dir = '/home/jakobs/Documents/IndEcol/Data/ecoinvent/ecoinvent 3.5_cutoff_ecoSpold02/'
# project_name = 'ecoinvent_35_cutoff'
# out_dir = '/home/jakobs/Documents/IndEcol/Data/ecospold2matrix_matrices/'

# parser = e2m.Ecospold2Matrix(ecospold_dir, project_name, out_dir, positive_waste=True, nan2null=True,
#                             characterisation_file=
#         '/home/jakobs/data/ecoinvent/ecoinvent 3.5_LCIA_implementation/LCIA_implementation_3.5.xlsx', )
# parser.ecospold_to_Leontief(fileformats='Pandas',with_absolute_flows=False)

## Load ecoinvent pickle files

In [2]:
%%time
with gzip.open('/home/jakobs/Documents/IndEcol/Data/ecospold2matrix_matrices/ecoinvent_35_cutoffPandas_symmNorm.gz.pickle','rb') as f:
    ecoinvent = pd.read_pickle(f)

CPU times: user 7.78 s, sys: 440 ms, total: 8.22 s
Wall time: 8.23 s


### Imput missing ecoinvent price data
Ecospold2matrix seems to have a bug and not extract all price data. However, the datasheets in filter.xlsx contains the actual ecoinvent prices for the processes so manually imput them into the PRO dataframe

In [3]:
file =  pd.ExcelFile('/home/jakobs/Documents/IndEcol/OASES/pylcaio/src/Data/eco3.5_exio3/Filter.xlsx')
sheetnames = file.sheet_names

for sheet_name in sheetnames:
    if not sheet_name == 'No price':
        print(sheet_name)
        sheet = file.parse(sheet_name, index_col=0)
        #find UUID of those processes that do not have a price (NaN)
        indices = list(set(ecoinvent['PRO'].loc[ecoinvent['PRO'].price.isnull()].index.tolist()).intersection(
                  sheet.index.tolist()))
        ecoinvent['PRO'].loc[indices, 'price'] = sheet.loc[indices, 'Price']
        ecoinvent['PRO'].loc[indices, 'priceUnit'] = 'EUR2005'

Hybridized
Market
Not commercialized
Poor quality
Empty processes
Uncovered geography


## Load Exiobase into a Dataframe with pymrio

In [4]:
%%time
EXIOBASE_path = '/home/jakobs/Documents/IndEcol/Data/EXIOBASE/exiobase3_6/IOT_2012_pxp'
io = pymrio.parse_exiobase3(EXIOBASE_path)

CPU times: user 35 s, sys: 924 ms, total: 36 s
Wall time: 36.2 s


In [ ]:
# path_to_capital_matrix = '/home/jakobs/Documents/IndEcol/Data/EXIOBASE/exiobase3_6/capital consumption matrices/Kbar_exio_v3_6_2012pxp.mat'

## Load aggregation dictionary

In [ ]:
# agg_dic = Aggregation_dic_from_file(os.path.join(os.path.dirname(os.path.dirname(EXIOBASE_path)),'Product_Country_aggregation.xlsx'))

## Read in data with pylcaio

In [5]:
%%time
database_loader = pylcaio.DatabaseLoader(ecoinvent, io, 'ecoinvent3.5', 'exiobase3')

CPU times: user 11.8 ms, sys: 124 µs, total: 11.9 ms
Wall time: 5.79 ms


In [6]:
%%time
lcaio_object = database_loader.combine_ecoinvent_exiobase(complete_extensions=False, 
                                        impact_world=False, regionalized=False)

No path for the capital folder was provided. Capitals will not be endogenized
CPU times: user 2min 49s, sys: 5.39 s, total: 2min 54s
Wall time: 1min 47s


In [ ]:
# lcaio_object.aggregate(agg_dic=agg_dic, build_aggregation_matrix_function=build_MultiIndex_Aggregation_Matrix)

In [7]:
%%time
lcaio_object.hybridize(price_neutral_cut_off_matrix=False, capitals=False, priceless_scaling=False, calculate_cutoff_variance=False)

Indentifying Rest of World regions...
Updating electricity prices...
Calculating productions volumes...
Adjusting low production volume processes...
Extending inventory...
Building H matrix...
Building geography concordance...
Building geographic conconcordance took 4.729621171951294 seconds
Filter H matrix...
Build Cut-off matrix...
CPU times: user 2min 16s, sys: 11.6 s, total: 2min 28s
Wall time: 2min 24s


In [8]:
%%time
# lcaio_object.variance_flag = False
lcaio_object.correct_double_counting('STAM', capitals=False)

STAM double counting correction being appliedi...
... Calculating Lambda Filter Matrix...
... Calculating Gamma Filter Matrix...
... Calculating Phi Filter Matrix...
Correcting cut-off matrices...
CPU times: user 7min 17s, sys: 13.4 s, total: 7min 31s
Wall time: 5min 20s


In [10]:
%%time
file_name_path = '/home/jakobs/Documents/IndEcol/OASES/Accuracy_vs_precission_HLCA/Databases/plcaio_object_STAM.pickle'
with gzip.open(file_name_path, 'wb') as fh:
    pickle.dump(lcaio_object, fh)
print('lcaio_object written to" {}'.format(file_name_path))

lcaio_object written to" /home/jakobs/Documents/IndEcol/OASES/Accuracy_vs_precission_HLCA/Databases/plcaio_object_STAM.pickle
CPU times: user 4min 58s, sys: 2.42 s, total: 5min
Wall time: 5min


In [ ]:
# %%time
# lcaio_object.correct_double_counting('binary', capitals=False)

In [9]:
%%time
lcaio_object.save_system(file_path='/home/jakobs/Documents/IndEcol/OASES/TEST_PYLCAIO/Databases/', file_name='extend_inventory_fixed-absolute-noise-nulling_2012_STAM', format='pickle')

Database saved to /home/jakobs/Documents/IndEcol/OASES/TEST_PYLCAIO/Databases/extend_inventory_fixed-absolute-noise-nulling_2012_STAM
Description file saved to /home/jakobs/Documents/IndEcol/OASES/TEST_PYLCAIO/Databases/description_extend_inventory_fixed-absolute-noise-nulling_2012_STAM.txt
CPU times: user 22.3 s, sys: 316 ms, total: 22.6 s
Wall time: 22.5 s


In [49]:
def powerseriessum(a,n=2):
    for i in range(n+1):
        if i==0:
            b = a
        else:
            b += b.dot(a)
    return scipy.sparse.csr_matrix(np.eye(a.shape[0])) + b

In [53]:
%%time
scipy.sparse.csr_matrix(np.eye(lcaio_object.A_ff.shape[0])) + lcaio_object.A_ff

CPU times: user 1.98 s, sys: 249 ms, total: 2.23 s
Wall time: 2.27 s


<16022x16022 sparse matrix of type '<class 'numpy.float64'>'
	with 173660 stored elements in Compressed Sparse Row format>

In [52]:
%%time
powerseriessum(lcaio_object.A_ff, n=6)

CPU times: user 13.9 s, sys: 2.6 s, total: 16.5 s
Wall time: 16.9 s


<16022x16022 sparse matrix of type '<class 'numpy.float64'>'
	with 107603641 stored elements in Compressed Sparse Row format>

In [48]:
%%time
lcaio_object.A_ff.dot(lcaio_object.A_ff)

CPU times: user 15.1 ms, sys: 0 ns, total: 15.1 ms
Wall time: 15.3 ms


<16022x16022 sparse matrix of type '<class 'numpy.float64'>'
	with 1077116 stored elements in Compressed Sparse Row format>

In [ ]:
%%time
scipy.sparse.linalg.spsolve(scipy.sparse.csr_matrix(np.eye(lcaio_object.A_ff.shape[0]))-lcaio_object.A_ff, np.eye(lcaio_object.A_ff.shape[0]))

In [19]:
f_38 = pd.read_csv('/home/jakobs/Documents/IndEcol/Data/EXIOBASE/EXIOBASE_3_8_2/IOT_2015_pxp/satellite/F.txt', sep='\t', skiprows=[2], header=[0,1], skipfooter=9, index_col=0)

/home/jakobs/anaconda3/envs/pylcaio_paradiso/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [20]:
f_38

region                                                     AT             \
sector                                             Paddy rice      Wheat   
Taxes less subsidies on products purchased: Total         0.0   4.946780   
Other net taxes on production                             0.0 -14.102623   
Compensation of employees; wages, salaries, & e...        0.0   0.926845   
Compensation of employees; wages, salaries, & e...        0.0  12.634327   
Compensation of employees; wages, salaries, & e...        0.0   2.033672   
...                                                       ...        ...   
Water Withdrawal Blue - Electricity - once-thro...        0.0   0.000000   
Water Withdrawal Blue - Electricity - once-thro...        0.0   0.000000   
Water Withdrawal Blue - Electricity - once-thro...        0.0   0.000000   
Water Withdrawal Blue - Electricity - once-thro...        0.0   0.000000   
Water Withdrawal Blue - Domestic - domestic Wat...        0.0   0.000000   

region                                                                \
sector                                             Cereal grains nec   
Taxes less subsidies on products purchased: Total           9.889373   
Other net taxes on production                             -31.989202   
Compensation of employees; wages, salaries, & e...          1.936045   
Compensation of employees; wages, salaries, & e...         26.391284   
Compensation of employees; wages, salaries, & e...          4.248047   
...                                                              ...   
Water Withdrawal Blue - Electricity - once-thro...          0.000000   
Water Withdrawal Blue - Electricity - once-thro...          0.000000   
Water Withdrawal Blue - Electricity - once-thro...          0.000000   
Water Withdrawal Blue - Electricity - once-thro...          0.000000   
Water Withdrawal Blue - Domestic - domestic Wat...          0.000000   

region                                                                      \
sector                                             Vegetables, fruit, nuts   
Taxes less subsidies on products purchased: Total                 9.534065   
Other net taxes on production                                   -43.360242   
Compensation of employees; wages, salaries, & e...                3.094868   
Compensation of employees; wages, salaries, & e...               42.187827   
Compensation of employees; wages, salaries, & e...                6.790722   
...                                                                    ...   
Water Withdrawal Blue - Electricity - once-thro...                0.000000   
Water Withdrawal Blue - Electricity - once-thro...                0.000000   
Water Withdrawal Blue - Electricity - once-thro...                0.000000   
Water Withdrawal Blue - Electricity - once-thro...                0.000000   
Water Withdrawal Blue - Domestic - domestic Wat...                0.000000   

region                                                        \
sector                                             Oil seeds   
Taxes less subsidies on products purchased: Total   1.056531   
Other net taxes on production                      -6.768552   
Compensation of employees; wages, salaries, & e...  0.217073   
Compensation of employees; wages, salaries, & e...  2.959040   
Compensation of employees; wages, salaries, & e...  0.476299   
...                                                      ...   
Water Withdrawal Blue - Electricity - once-thro...  0.000000   
Water Withdrawal Blue - Electricity - once-thro...  0.000000   
Water Withdrawal Blue - Electricity - once-thro...  0.000000   
Water Withdrawal Blue - Electricity - once-thro...  0.000000   
Water Withdrawal Blue - Domestic - domestic Wat...  0.000000   

region                                                                     \
sector                                             Sugar cane, sugar beet   
Taxes less subsidies on products purchased: Total                0.81066

In [21]:
f_34 = pd.read_csv('/home/jakobs/Documents/IndEcol/Data/EXIOBASE/exiobase_3_4_IOT_pxp/IOT_2011_pxp/satellite/F.txt', sep='\t', header=[0,1], index_col=0)

In [28]:
f_34.index[~(f_34.index == f_38.index)]

Index(['Compensation of employees; wages, salaries, & employers social contributions: Low-skilled',
       'Compensation of employees; wages, salaries, & employers social contributions: Medium-skilled',
       'Compensation of employees; wages, salaries, & employers social contributions: High-skilled'],
      dtype='object')

In [30]:
f_38.index[~(f_34.index == f_38.index)]

Index(['Compensation of employees; wages, salaries, & employers' social contributions: Low-skilled',
       'Compensation of employees; wages, salaries, & employers' social contributions: Medium-skilled',
       'Compensation of employees; wages, salaries, & employers' social contributions: High-skilled'],
      dtype='object')

In [31]:
%%time
EXIOBASE_path = '/home/jakobs/Documents/IndEcol/Data/EXIOBASE/EXIOBASE_3_8_2/IOT_2015_pxp/'
io = pymrio.parse_exiobase3(EXIOBASE_path)

CPU times: user 1min 33s, sys: 1.71 s, total: 1min 34s
Wall time: 1min 36s


In [34]:
io.satellite.F.drop(io.satellite.F.index[-9:], inplace=True)

region                                                     AT             \
sector                                             Paddy rice      Wheat   
stressor                                                                   
Taxes less subsidies on products purchased: Total         0.0   4.946780   
Other net taxes on production                             0.0 -14.102623   
Compensation of employees; wages, salaries, & e...        0.0   0.926845   
Compensation of employees; wages, salaries, & e...        0.0  12.634327   
Compensation of employees; wages, salaries, & e...        0.0   2.033672   
...                                                       ...        ...   
Water Withdrawal Blue - Electricity - once-thro...        0.0   0.000000   
Water Withdrawal Blue - Electricity - once-thro...        0.0   0.000000   
Water Withdrawal Blue - Electricity - once-thro...        0.0   0.000000   
Water Withdrawal Blue - Electricity - once-thro...        0.0   0.000000   
Water Withdrawal Blue - Domestic - domestic Wat...        0.0   0.000000   

region                                                                \
sector                                             Cereal grains nec   
stressor                                                               
Taxes less subsidies on products purchased: Total           9.889373   
Other net taxes on production                             -31.989202   
Compensation of employees; wages, salaries, & e...          1.936045   
Compensation of employees; wages, salaries, & e...         26.391284   
Compensation of employees; wages, salaries, & e...          4.248047   
...                                                              ...   
Water Withdrawal Blue - Electricity - once-thro...          0.000000   
Water Withdrawal Blue - Electricity - once-thro...          0.000000   
Water Withdrawal Blue - Electricity - once-thro...          0.000000   
Water Withdrawal Blue - Electricity - once-thro...          0.000000   
Water Withdrawal Blue - Domestic - domestic Wat...          0.000000   

region                                                                      \
sector                                             Vegetables, fruit, nuts   
stressor                                                                     
Taxes less subsidies on products purchased: Total                 9.534065   
Other net taxes on production                                   -43.360242   
Compensation of employees; wages, salaries, & e...                3.094868   
Compensation of employees; wages, salaries, & e...               42.187827   
Compensation of employees; wages, salaries, & e...                6.790722   
...                                                                    ...   
Water Withdrawal Blue - Electricity - once-thro...                0.000000   
Water Withdrawal Blue - Electricity - once-thro...                0.000000   
Water Withdrawal Blue - Electricity - once-thro...                0.000000   
Water Withdrawal Blue - Electricity - once-thro...                0.000000   
Water Withdrawal Blue - Domestic - domestic Wat...                0.000000   

region                                                        \
sector                                             Oil seeds   
stressor                                                       
Taxes less subsidies on products purchased: Total   1.056531   
Other net taxes on production                      -6.768552   
Compensation of employees; wages, salaries, & e...  0.217073   
Compensation of employees; wages, salaries, & e...  2.959040   
Compensation of employees; wages, salaries, & e...  0.476299   
...                                                      ...   
Water Withdrawal Blue - Electricity - once-thro...  0.000000   
Water Withdrawal Blue - Electricity - once-thro...  0.000000   
Water Withdrawal Blue - Electricity - once-thro...  0.000000   
Water Withdrawal Blue - Electricity - once-thro...  0.000000   
W

In [35]:
io.satellite.F.index[-9:]

Index(['Energy Carrier Net Total', 'Energy Carrier Net NENE',
       'Energy Carrier Net NTRA', 'Energy Carrier Net TAVI',
       'Energy Carrier Net TMAR', 'Energy Carrier Net TOTH',
       'Energy Carrier Net TRAI', 'Energy Carrier Net TROA',
       'Energy Carrier Net LOSS'],
      dtype='object', name='stressor')